In [1]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk import FreqDist
import re

# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located on Kaggle (https://www.kaggle.com/kazanova/sentiment140). Once you have downloaded and imported the dataset, it you will need to define the columns names: df.columns = ['target','id','date','flag','user','text']

*Notes:* 

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [29]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ines_\AppData\Roaming\nltk_data...


True

In [30]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

txt = "Ironhack is a Global Tech School ranked num 2 worldwide. Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
analyzer = SentimentIntensityAnalyzer()
sentiment_scores = analyzer.polarity_scores(txt)

print(sentiment_scores)

{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}


In [31]:
import pandas as pd
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding='latin1', header=None)
data.columns = ['target', 'id', 'date', 'flag', 'user', 'text']
subset_data = data.sample(n=20000, random_state=42)
subset_data

,target,id,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem
...,...,...,...,...,...,...
1178000,4,1981517014,Sun May 31 09:19:12 PDT 2009,NO_QUERY,gemmaridlington,Loving the weather Although we're not having ...
667014,0,2245469775,Fri Jun 19 16:10:38 PDT 2009,NO_QUERY,musicroxmysockz,Me and andrea r. last day of school..... http...
1451235,4,2063022808,Sun Jun 07 01:05:46 PDT 2009,NO_QUERY,ElaineToni,Just finished watching Your Song Presents: Boy...
1181413,4,1982082859,Sun May 31 10:29:36 PDT 2009,NO_QUERY,lindseyrd20,@janfran813 awww i can't wait to get one


### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [32]:
import re

def clean_up(s):
    # Remove numbers
    s = re.sub(r'\d', '', s)

    # Remove URLs
    s = re.sub(r'http\S+|www\S+', '', s)

    # Remove special characters and convert to lowercase
    s = re.sub(r"[!?,:.;&@#()\[\]{}'-]", ' ', s).lower()

    # Remove extra spaces and strip leading/trailing spaces
    s = re.sub(r'\s+', ' ', s).strip()

    return s

subset_data['cleaned'] = subset_data['text'].apply(clean_up)

In [33]:
subset_data.head()

,target,id,date,flag,user,text,cleaned
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,chrishasboobs ahhh i hope your ok
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...",misstoriblack cool i have no tweet apps for my...
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,tiannachaos i know just family drama its lame ...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,school email won t open and i have geography s...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,upper airways problem


In [34]:
import nltk
from nltk.tokenize import word_tokenize

def tokenize(s):
    tokens = word_tokenize(s)
    return tokens

subset_data['tokenized'] = subset_data['cleaned'].apply(tokenize)

In [35]:
subset_data.head()

,target,id,date,flag,user,text,cleaned,tokenized
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,chrishasboobs ahhh i hope your ok,"[chrishasboobs, ahhh, i, hope, your, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...",misstoriblack cool i have no tweet apps for my...,"[misstoriblack, cool, i, have, no, tweet, apps..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,tiannachaos i know just family drama its lame ...,"[tiannachaos, i, know, just, family, drama, it..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,school email won t open and i have geography s...,"[school, email, won, t, open, and, i, have, ge..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,upper airways problem,"[upper, airways, problem]"


In [36]:
import nltk
from nltk.stem import SnowballStemmer

nltk.download('punkt')

def stem_and_lemmatize(word_list):
    stemmer = SnowballStemmer("english")
    stemmed_and_lemmatized = []
    
    for word in word_list:
        stemmed_word = stemmer.stem(word)
        stemmed_and_lemmatized.append(stemmed_word)
    
    return stemmed_and_lemmatized

subset_data['word_list'] = subset_data['tokenized'].apply(stem_and_lemmatize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ines_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [37]:
subset_data.head()

,target,id,date,flag,user,text,cleaned,tokenized,word_list
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,chrishasboobs ahhh i hope your ok,"[chrishasboobs, ahhh, i, hope, your, ok]","[chrishasboob, ahhh, i, hope, your, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...",misstoriblack cool i have no tweet apps for my...,"[misstoriblack, cool, i, have, no, tweet, apps...","[misstoriblack, cool, i, have, no, tweet, app,..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,tiannachaos i know just family drama its lame ...,"[tiannachaos, i, know, just, family, drama, it...","[tiannachao, i, know, just, famili, drama, it,..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,school email won t open and i have geography s...,"[school, email, won, t, open, and, i, have, ge...","[school, email, won, t, open, and, i, have, ge..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,upper airways problem,"[upper, airways, problem]","[upper, airway, problem]"


In [38]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

def remove_stopwords(word_list):
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in word_list if word not in stop_words]
    return filtered_words

subset_data['text_processed'] = subset_data['word_list'].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ines_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
subset_data.drop(['cleaned', 'tokenized', 'word_list'], axis=1, inplace=True)

In [40]:
subset_data.head()

,target,id,date,flag,user,text,text_processed
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!,"[chrishasboob, ahhh, hope, ok]"
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo...","[misstoriblack, cool, tweet, app, razr]"
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...,"[tiannachao, know, famili, drama, lame, hey, n..."
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...,"[school, email, open, geographi, stuff, revis,..."
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem,"[upper, airway, problem]"


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [41]:
from nltk import FreqDist

all_words = [word for word_list in subset_data['text_processed'] for word in word_list]

freq_dist = FreqDist(all_words)

top_words = freq_dist.most_common(5000)

top_words = [word[0] for word in top_words]

print(top_words)

['go', 'get', 'day', 'good', 'work', 'like', 'love', 'quot', 'today', 'got', 'time', 'lol', 'thank', 'want', 'one', 'know', 'miss', 'back', 'feel', 'realli', 'see', 'u', 'think', 'amp', 'im', 'hope', 'well', 'watch', 'still', 'night', 'make', 'oh', 'need', 'na', 'look', 'home', 'come', '*', 'new', 'last', 'much', 'twitter', 'great', 'morn', 'wish', 'sleep', 'wait', 'follow', 'sad', 'tomorrow', 'tri', 'haha', 'bad', 'week', 'sorri', 'say', 'onli', 'happi', 'veri', 'would', 'right', 'whi', 'fun', 'friend', 'tonight', 'thing', 'nice', 'better', 'though', 'take', 'even', 'start', 'bed', 'hate', 'gon', 'peopl', 'way', 'tweet', 'school', 'yeah', 'show', 'lt', 'awesom', 'could', 'play', 'final', 'dont', 'hour', 'everyon', 'first', 'guy', 'use', 'next', 'let', 'weekend', 'year', 'soon', 'littl', 'pleas', 'long', 'yes', 'never', 'wan', 'rain', 'hey', 'girl', 'find', 'cant', 'help', 'ok', 'man', 'movi', 'done', 'best', 'sure', 'tire', 'bore', 'sick', 'x', 'ani', 'eat', 'alway', 'us', 'alreadi', 

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [42]:
import numpy as np
import pandas as pd

features = np.zeros((len(top_words), len(subset_data)), dtype=bool)

for i, words_list in enumerate(top_words):
    for j, word in enumerate(top_words):
        if word in words_list:
            features[j, i] = True

feature_data = {word: features[i, :] for i, word in enumerate(top_words)}
features = pd.DataFrame(feature_data)

In [43]:
features.head()

,go,get,day,good,work,like,love,quot,today,got,...,nest,lodg,racist,devill,jungl,greekpeac,bks,aidan,photogen,municip
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

def get_sentiment(text):
    scores = sia.polarity_scores(text)
    compound_score = scores['compound']
    return compound_score > 0

features['is_positive'] = features.apply(lambda row: get_sentiment(' '.join(row.index[row])), axis=1)
features

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ines_\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,go,get,day,good,work,like,love,quot,today,got,...,lodg,racist,devill,jungl,greekpeac,bks,aidan,photogen,municip,is_positive
0,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [45]:
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier

train_set, test_set = train_test_split(features, test_size=0.2, random_state=42)
train_data = [(row.to_dict(), row['is_positive']) for _, row in train_set.iterrows()]

classifier = NaiveBayesClassifier.train(train_data)
classifier.show_most_informative_features()

Most Informative Features
                     cha = True             True : False  =    739.7 : 1.0
                      ha = True             True : False  =    614.0 : 1.0
                      ff = True             True : False  =    443.8 : 1.0
                      ok = True             True : False  =    288.6 : 1.0
                     tha = True             True : False  =    187.7 : 1.0
                     eff = True             True : False  =    143.5 : 1.0
                   right = True             True : False  =     99.4 : 1.0
                     lay = True             True : False  =     99.4 : 1.0
                      nj = True             True : False  =     77.3 : 1.0
                     hat = True             True : False  =     61.5 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [46]:
from nltk.classify import accuracy
test_data = [(row.to_dict(), row['is_positive']) for _, row in test_set.iterrows()]
accuracy_score = accuracy(classifier, test_data)
print("Accuracy:", accuracy_score)

Accuracy: 0.98775


## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here

## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here